# LangChain 0.3 中的 Chains（链）概念详解
> 在 LangChain 0.3 中，Chains（链）是一种将多个组件（如 LLM、提示模板、记忆等）串联起来形成完整应用流程的方式。虽然 LangChain 0.3 更推荐使用 LCEL（LangChain Expression Language）构建应用，但传统的 Chain 类仍然被支持。

传统 Chain 的主要类型
根据提供的文档，LangChain 0.3 中的传统 Chain 主要包括：
1. LLMChain - 最基础的链，结合提示模板和 LLM
2. ConversationChain - 带有记忆功能的对话链
3. ConversationalRetrievalQAChain - 用于与文档进行对话的链
4. StuffDocumentsChain - 将多个文档合并到一个提示中
5. MapReduceDocumentsChain - 先对每个文档单独处理，再合并结果
6. RefineDocumentsChain - 通过逐步细化答案处理多个文档
7. SqlDatabaseChain - 通过生成和执行 SQL 查询回答问题
8. MultiPromptChain - 在多个提示之间路由输入

## 示例 1: 基础 LLMChain

In [3]:
# 安装依赖
# pip install langchain langchain-ollama

from langchain_ollama import OllamaLLM
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

# 初始化 LLM
llm = OllamaLLM(model="qwen2.5:3b")

# 创建提示模板
prompt = PromptTemplate(
    input_variables=["topic"],
    template="请用简单的语言解释{topic}是什么？"
)

# 创建 LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    # verbose=True  # 显示执行过程
)

# 运行链
result = chain.run(topic="量子计算")
print(f"结果: {result}")

结果: 量子计算是一种基于量子力学原理的计算方式，利用了量子位（也叫量子比特或qubit）的独特性质来进行信息处理。在经典计算机中，信息是以0和1两种状态来表示的；而在量子计算机中，量子位可以同时处于0和1的状态，这是由于量子力学中的叠加态特性。

另外，还有一个特性叫做纠缠，意味着两个或多个量子位之间存在着一种特殊的关联，即使它们被分开很远。在量子计算中，可以通过处理这些叠加态和纠缠态来执行一些经典计算机无法完成的任务，尤其是那些涉及大量数据的复杂问题求解任务。

简单来说，量子计算利用了量子力学的奇妙现象来存储和操作信息，这可能会让计算机的速度和效率大大提高。


## 示例 2: ConversationChain 实现对话记忆

In [6]:
from langchain_ollama import OllamaLLM
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 初始化 LLM
llm = OllamaLLM(model="qwen2.5:3b",num_predict=10)

# 创建记忆
memory = ConversationBufferMemory()

# 创建对话链
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    # verbose=True
)

# 多轮对话测试
print("对话测试:")
response1 = conversation.predict(input="你好，我叫张三，是一名程序员")
print(f"AI: {response1}")

response2 = conversation.predict(input="你还记得我的名字和职业吗？")
print(f"AI: {response2}")

# 查看记忆内容
print(f"\n记忆内容: {memory.buffer}")

对话测试:
AI: 你好张先生，很高兴认识你！作为你的助
AI: 当然记得，您叫张三，是一名程序员

记忆内容: Human: 你好，我叫张三，是一名程序员
AI: 你好张先生，很高兴认识你！作为你的助
Human: 你还记得我的名字和职业吗？
AI: 当然记得，您叫张三，是一名程序员


## 示例 3: 使用不同类型的记忆

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferWindowMemory,
    ConversationSummaryMemory
)

# 初始化 LLM
llm = OllamaLLM(model="llama2")

# 1. 窗口记忆 - 只保留最近k轮对话
print("=" * 50)
print("窗口记忆 (只保留最近2轮对话)")
print("=" * 50)

window_memory = ConversationBufferWindowMemory(k=2)
window_chain = ConversationChain(
    llm=llm,
    memory=window_memory,
    verbose=True
)

# 测试多轮对话
conversations = [
    "我叫李四",
    "我住在北京",
    "我是医生",
    "我的名字是什么？",  # 应该记不住，因为超出窗口
    "我的职业是什么？"  # 应该记得住
]

for i, msg in enumerate(conversations, 1):
    print(f"\n第{i}轮对话:")
    response = window_chain.predict(input=msg)
    print(f"人类: {msg}")
    print(f"AI: {response}")

# 2. 摘要记忆 - 自动总结对话历史
print("\n" + "=" * 50)
print("摘要记忆")
print("=" * 50)

summary_memory = ConversationSummaryMemory(llm=llm)
summary_chain = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)

# 测试对话
print("\n摘要记忆测试:")
response1 = summary_chain.predict(input="我是王五，来自上海")
print(f"AI: {response1}")

response2 = summary_chain.predict(input="我喜欢人工智能和机器学习")
print(f"AI: {response2}")

response3 = summary_chain.predict(input="你能告诉我我来自哪里吗？")
print(f"AI: {response3}")

# 查看摘要内容
print(f"\n摘要内容: {summary_memory.moving_summary_buffer}")

## 示例 4: 自定义提示模板的 LLMChain

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# 初始化 LLM
llm = OllamaLLM(model="llama2")

# 创建自定义提示模板
custom_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template="""你是一个友好的AI助手，能够记住对话历史。

对话历史:
{history}

人类: {input}
AI助手:"""
)

# 创建记忆
custom_memory = ConversationBufferMemory(memory_key="history")

# 创建LLM链
custom_chain = LLMChain(
    llm=llm,
    prompt=custom_prompt,
    memory=custom_memory,
    verbose=True
)

# 测试对话
print("\n测试对话:")
response1 = custom_chain.invoke("我喜欢看科幻电影，特别是《星际穿越》")
print(f"AI: {response1['text']}")

response2 = custom_chain.invoke("我刚才提到了什么电影？")
print(f"AI: {response2['text']}")

## 示例 5: 基于令牌数量的记忆限制

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory

# 初始化 LLM
llm = OllamaLLM(model="llama2")

# 创建基于令牌的记忆 (限制最大令牌数)
token_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=100  # 设置最大令牌数
)

# 创建对话链
token_chain = ConversationChain(
    llm=llm,
    memory=token_memory,
    verbose=True
)

# 测试对话
print("\n基于令牌的记忆测试:")
response1 = token_chain.predict(input="我是一名软件工程师，专门从事人工智能和机器学习的研究与开发")
print(f"AI: {response1}")

response2 = token_chain.predict(input="我目前在研究大语言模型的应用，特别关注对话系统的优化")
print(f"AI: {response2}")

response3 = token_chain.predict(input="你知道我的专业领域吗？")
print(f"AI: {response3}")

## 示例 6: 使用 StuffDocumentsChain 处理多个文档

In [7]:
from langchain_ollama import OllamaLLM
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

# 初始化 LLM
llm = OllamaLLM(model="qwen2.5:3b")

# 创建文档提示模板
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)

# 创建提示模板
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""根据以下文档内容回答问题:

{context}

问题: {question}
回答:"""
)

# 创建 LLM 链
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 创建 StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    verbose=True
)

# 创建示例文档
documents = [
    Document(page_content="Python是一种高级编程语言，以简洁、易读的语法著称。"),
    Document(page_content="Python支持多种编程范式，包括面向对象、命令式和函数式编程。"),
    Document(page_content="Python由Guido van Rossum于1991年创建，现在由Python软件基金会管理。")
]

# 运行链
result = stuff_chain.invoke({
    "question": "Python是什么语言？它有什么特点？",
    "input_documents": documents
})

print(f"结果: {result['output_text']}")

/var/folders/8k/msfr41dj559byywk1kcjpm9h0000gn/T/ipykernel_2402/413710473.py:30: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(




> Entering new StuffDocumentsChain chain...

> Finished chain.
结果: Python是一种高级编程语言。它的主要特点包括：

1. 简洁：Python的语法设计简洁明了，这使得代码更易读。
2. 易读性：Python语法易于理解和阅读，这对于快速编写和理解程序非常重要。
3. 支持多种编程范式：Python兼容面向对象、命令式以及函数式编程风格。
4. 由Guido van Rossum于1991年创建，并且现在由Python软件基金会进行管理。

以上就是Python的特点。


## 示例 7: 使用 MapReduceDocumentsChain 处理大量文档

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.chains import MapReduceDocumentsChain, LLMChain, ReduceDocumentsChain
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

# 初始化 LLM
llm = OllamaLLM(model="llama2")

# Map 提示模板 - 处理单个文档
map_template = """以下是一段关于Python的文本:
{document}

提取这段文本中关于Python的重要信息，并简洁总结:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce 提示模板 - 合并多个文档的结果
reduce_template = """以下是多段关于Python的总结:
{doc_summaries}

将这些信息整合成一个连贯的段落，回答问题: {question}
最终答案:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# 创建 ReduceDocumentsChain
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=reduce_chain,
    collapse_documents_chain=reduce_chain,
    token_max=4000,
)

# 创建 MapReduceDocumentsChain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="document",
    return_intermediate_steps=True,
    verbose=True
)

# 创建示例文档
documents = [
    Document(page_content="Python是一种高级编程语言，以简洁、易读的语法著称。"),
    Document(page_content="Python支持多种编程范式，包括面向对象、命令式和函数式编程。"),
    Document(page_content="Python由Guido van Rossum于1991年创建，现在由Python软件基金会管理。"),
    Document(page_content="Python的设计哲学强调代码的可读性和简洁的语法，使开发者能够用更少的代码表达想法。"),
    Document(page_content="Python有丰富的标准库，被称为'自带电池'的语言。")
]

# 运行链
result = map_reduce_chain.invoke({
    "input_documents": documents,
    "question": "Python是什么，它有哪些主要特点？"
})

print("\n最终结果:")
print(result["output_text"])

print("\n中间步骤:")
for i, step in enumerate(result["intermediate_steps"], 1):
    print(f"步骤 {i}: {step}")

## 示例 8: 使用 RefineDocumentsChain 逐步细化答案

In [9]:
from langchain_ollama import OllamaLLM
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

# 初始化 LLM
llm = OllamaLLM(model="qwen2.5:3b")

# 初始提示模板 - 基于第一个文档生成初始答案
initial_prompt = PromptTemplate.from_template(
    "根据以下信息回答问题: {question}\n\n{document}\n\n回答:"
)
initial_chain = LLMChain(llm=llm, prompt=initial_prompt)

# 细化提示模板 - 基于更多文档细化答案
refine_prompt = PromptTemplate.from_template(
    """根据原始问题和之前的答案，结合新的文档内容，改进你的答案。

问题: {question}
之前的答案: {existing_answer}
新文档内容: {document}

改进后的答案:"""
)
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

# 创建 RefineDocumentsChain
refine_documents_chain = RefineDocumentsChain(
    initial_llm_chain=initial_chain,
    refine_llm_chain=refine_chain,
    document_variable_name="document",
    verbose=True
)

# 创建示例文档
documents = [
    Document(page_content="机器学习是人工智能的一个子领域，专注于开发能够从数据中学习的算法。"),
    Document(page_content="监督学习是机器学习的一种类型，其中模型从带标签的训练数据中学习。"),
    Document(page_content="无监督学习是另一种机器学习类型，模型从没有标签的数据中发现模式。"),
    Document(page_content="深度学习是机器学习的一个子集，使用多层神经网络处理复杂任务。"),
    Document(page_content="强化学习是一种机器学习方法，智能体通过与环境交互来学习最佳行为。")
]

# 运行链
result = refine_documents_chain.invoke({
    "input_documents": documents,
    "question": "什么是机器学习及其主要类型？"
})

print(f"最终结果: {result['output_text']}")

ValidationError: 1 validation error for RefineDocumentsChain
initial_response_name
  Field required [type=missing, input_value={'initial_llm_chain': LLM...ument', 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

## 示例 9: 使用 ConstitutionalChain 实现有原则的回答

In [11]:
from langchain_ollama import OllamaLLM
from langchain.chains import ConstitutionalChain, LLMChain
from langchain_core.prompts import PromptTemplate

# 初始化 LLM
llm = OllamaLLM(model="qwen2.5:3b")

# 创建基础链
prompt = PromptTemplate(
    template="请回答以下问题: {question}",
    input_variables=["question"]
)
chain = LLMChain(llm=llm, prompt=prompt)

# 定义宪法原则
constitutional_principles = [
    {
        "name": "科学准确性",
        "critique_request": "评估回答是否基于科学事实，避免误导性信息。",
        "revision_request": "修改回答，确保科学准确性，避免任何误导性陈述。"
    },
    {
        "name": "平衡观点",
        "critique_request": "评估回答是否提供了平衡的观点，避免偏见。",
        "revision_request": "修改回答，确保提供平衡的观点，考虑不同角度。"
    }
]

# 创建宪法链
constitutional_chain = ConstitutionalChain.from_llm(
    chain=chain,
    constitutional_principles=constitutional_principles,
    llm=llm,
    verbose=True
)

# 测试链
question = "全球变暖是真实的吗？"
result = constitutional_chain.invoke({"question": question})

print(f"最终回答: {result['output']}")



> Entering new ConstitutionalChain chain...
Initial response: 全球变暖是一个被广泛科学研究和证实的现象。自工业革命以来，人类活动尤其是化石燃料的燃烧（如煤炭、石油和天然气）大大增加了大气中的温室气体浓度，特别是二氧化碳。这导致地球表面平均温度在过去一个世纪中显著上升，从而引发了全球气候系统的变化。

科学家们通过各种数据来源和模型分析得出了这一结论，包括冰芯样本、海床沉积物以及长期的气温观测记录等。这些证据共同表明了人类活动对气候变化的影响，并且已经引起了全球范围内的环境变化，如极端天气事件增多、极地冰盖融化加速及海平面上升等。

尽管存在一些关于气候模型预测精确性的讨论或不确定性，但全球变暖的整体趋势和主要驱动力是被广泛认可的。面对这一挑战，国际社会正在积极寻求减少温室气体排放以及适应气候变化的方法，以保护地球环境并保障未来世代的生活质量。


> Finished chain.
最终回答: 全球变暖是一个被广泛科学研究和证实的现象。自工业革命以来，人类活动尤其是化石燃料的燃烧（如煤炭、石油和天然气）大大增加了大气中的温室气体浓度，特别是二氧化碳。这导致地球表面平均温度在过去一个世纪中显著上升，从而引发了全球气候系统的变化。

科学家们通过各种数据来源和模型分析得出了这一结论，包括冰芯样本、海床沉积物以及长期的气温观测记录等。这些证据共同表明了人类活动对气候变化的影响，并且已经引起了全球范围内的环境变化，如极端天气事件增多、极地冰盖融化加速及海平面上升等。

尽管存在一些关于气候模型预测精确性的讨论或不确定性，但全球变暖的整体趋势和主要驱动力是被广泛认可的。面对这一挑战，国际社会正在积极寻求减少温室气体排放以及适应气候变化的方法，以保护地球环境并保障未来世代的生活质量。


## 示例 10: 使用 MultiPromptChain 路由到不同专家

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.chains import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

# 初始化 LLM
llm = OllamaLLM(model="qwen2.5:3b")

# 定义不同领域的提示模板
prompt_infos = [
    {
        "name": "编程",
        "description": "处理编程和软件开发相关问题",
        "prompt_template": "你是一位编程专家。请回答以下编程问题:\n\n{input}"
    },
    {
        "name": "历史",
        "description": "处理历史相关问题",
        "prompt_template": "你是一位历史学家。请回答以下历史问题:\n\n{input}"
    },
    {
        "name": "科学",
        "description": "处理科学相关问题",
        "prompt_template": "你是一位科学家。请回答以下科学问题:\n\n{input}"
    }
]

# 创建路由提示
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=", ".join([p["name"] for p in prompt_infos]),
    destinations_and_descriptions="\n".join(
        [f"{p['name']}: {p['description']}" for p in prompt_infos]
    )
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

# 创建路由链
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 创建目标链
destination_chains = {}
for p_info in prompt_infos:
    prompt = PromptTemplate(template=p_info["prompt_template"], input_variables=["input"])
    destination_chains[p_info["name"]] = LLMChain(llm=llm, prompt=prompt)

# 创建默认链
default_prompt = PromptTemplate(
    template="你是一位通用知识专家。请回答以下问题:\n\n{input}",
    input_variables=["input"]
)
default_chain = LLMChain(llm=llm, prompt=default_prompt)

# 创建 MultiPromptChain
multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

# 测试不同类型的问题
questions = [
    "Python中如何创建一个列表？",
    "第二次世界大战是什么时候开始的？",
    "光速是多少？",
    "如何做一道美味的披萨？"  # 应该走默认链
]

for i, question in enumerate(questions, 1):
    print(f"\n问题 {i}: {question}")
    result = multi_prompt_chain.invoke({"input": question})
    print(f"回答: {result['text']}")

# 总结

## 传统 Chain 的优缺点

### 优点
1. **结构清晰** - 预定义的链结构使得应用逻辑清晰
2. **易于使用** - 对于简单场景，使用现成的链可以快速构建应用
3. **内置功能** - 许多链内置了特定功能，如记忆管理、文档处理等

### 缺点
1. **灵活性有限** - 相比 LCEL，传统链的自定义能力较弱
2. **可观察性较差** - 传统链的调试和监控能力不如 LCEL
3. **组合性较弱** - 难以像 LCEL 那样灵活组合不同组件

## 选择建议

1. **简单应用**: 使用 `LLMChain` 或 `ConversationChain`
2. **内存限制**: 使用带有 `ConversationBufferWindowMemory` 或 `ConversationTokenBufferMemory` 的链
3. **长对话**: 使用带有 `ConversationSummaryMemory` 或 `ConversationSummaryBufferMemory` 的链
4. **文档处理**: 根据文档数量和大小选择 `StuffDocumentsChain`、`MapReduceDocumentsChain` 或 `RefineDocumentsChain`
5. **复杂应用**: 考虑使用 LCEL 或 LangGraph 代替传统链

虽然 LangChain 0.3 更推荐使用 LCEL 构建应用，但传统链仍然是快速构建简单应用的有效工具。对于更复杂的应用，建议考虑使用 LCEL 或 LangGraph。
